In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

In [3]:
text = 'My cat is a great cat'
tokens = text.lower().split()
print('Words in the our text:', tokens)

Words in the our text: ['my', 'cat', 'is', 'a', 'great', 'cat']


In [4]:
vocab = set(tokens)
vocab = pd.Series(range(len(vocab)), index=vocab)
vocab

great    0
my       1
a        2
is       3
cat      4
dtype: int64

In [5]:
pd.get_dummies(tokens)

,a,cat,great,is,my
0,0,0,0,0,1
1,0,1,0,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,0,0,1,0,0
5,0,1,0,0,0


In [6]:
word_ids = vocab.loc[tokens].values
word_ids

array([1, 4, 3, 2, 0, 4], dtype=int64)

In [7]:
inputs = tf.placeholder(tf.int32, [None])

# TensorFlow has an operation for one-hot encoding
one_hot_inputs = tf.one_hot(inputs, len(vocab))

transformed = tf.Session().run(one_hot_inputs, {inputs: word_ids})
transformed

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [8]:
embedding_size = 3

inputs = tf.placeholder(tf.int32, [None], name='word_ids')

# This is where the embedding vectors live
# This will be modified by the optimization unless trainable=False
# I choose random normal distribution but you can try other distributions
embeddings = tf.random_normal(shape=(len(vocab), embedding_size))

# this will return the embedding lookup
embedded = tf.nn.embedding_lookup(embeddings, inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
[original,transformed] = sess.run([embeddings,embedded],{inputs: word_ids})
print(transformed)
print(original)

[[-0.2797505   0.566662   -1.2156818 ]
 [-0.73898786 -1.5640157  -1.5561099 ]
 [-0.8949445   0.43797335  1.4428164 ]
 [-0.5304903  -1.8031386  -0.25690705]
 [-0.31393436  0.8919702  -1.0153612 ]
 [-0.73898786 -1.5640157  -1.5561099 ]]
[[-0.31393436  0.8919702  -1.0153612 ]
 [-0.2797505   0.566662   -1.2156818 ]
 [-0.5304903  -1.8031386  -0.25690705]
 [-0.8949445   0.43797335  1.4428164 ]
 [-0.73898786 -1.5640157  -1.5561099 ]]


### Simple example for Keras

In [9]:
from numpy import array
from tensorflow import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!','Good work','Great effort',
        'nice work','Excellent!','Weak',
        'Poor effort!','not good','poor work','Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[4, 35], [42, 7], [43, 10], [5, 7], [26], [31], [28, 10], [38, 42], [28, 7], [29, 24, 35, 14]]
[[ 4 35  0  0]
 [42  7  0  0]
 [43 10  0  0]
 [ 5  7  0  0]
 [26  0  0  0]
 [31  0  0  0]
 [28 10  0  0]
 [38 42  0  0]
 [28  7  0  0]
 [29 24 35 14]]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
______________________________

Using TensorFlow backend.



Accuracy: 89.999998


### Simple example for TF
Param for embedding = 400 (50 vocabs * 8 embedding size)

In [10]:
labels = labels.reshape((-1,1))

FCW1 = weight_variable([32,1])
FCb1 = bias_variable([1])

targets = tf.placeholder(tf.float32,[None,1], name='labels')
inputs = tf.placeholder(tf.int32,[None,4], name='word_ids')
embeddings = tf.random_normal(shape=(50,8))
embedded = tf.nn.embedding_lookup(embeddings, inputs)
z = tf.reshape(embedded,[-1,4*8])
z = tf.matmul(z,FCW1) + FCb1
# define the loss function
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets,logits=z)
correct_prediction = tf.equal(tf.argmax(z, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# define training step and accuracy
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [12]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        train_accuracy = sess.run(accuracy,feed_dict={
                inputs:padded_docs,targets:labels})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        sess.run(train_step, feed_dict={inputs:padded_docs,
                                        targets:labels})

step 0, training accuracy 1
step 1, training accuracy 1
step 2, training accuracy 1
step 3, training accuracy 1
step 4, training accuracy 1
step 5, training accuracy 1
step 6, training accuracy 1
step 7, training accuracy 1
step 8, training accuracy 1
step 9, training accuracy 1
